In [0]:
USE CATALOG anac_ocorrencias_dev;


In [0]:
CREATE SCHEMA IF NOT EXISTS dimensional
MANAGED LOCATION 'abfss://gold@ocorrenciasanacdl.dfs.core.windows.net/dimensional'

In [0]:
CREATE TABLE IF NOT EXISTS anac_ocorrencias_dev.dimensional.dim_manufacturer (
  id_manufacturer BIGINT GENERATED ALWAYS AS IDENTITY NOT NULL,
  manufacturer_name STRING,
  CONSTRAINT pk_manufacturer PRIMARY KEY (id_manufacturer) RELY
);

MERGE INTO anac_ocorrencias_dev.dimensional.dim_manufacturer AS tgt
USING (
  SELECT DISTINCT
    TRIM(LOWER(acft_manufacturer)) AS manufacturer_trimmed,
    acft_manufacturer AS manufacturer_name
  FROM anac_ocorrencias_dev.gold.transformed_occurrences
  WHERE acft_manufacturer IS NOT NULL
) AS src
ON LOWER(TRIM(tgt.manufacturer_name)) = src.manufacturer_trimmed
WHEN NOT MATCHED THEN
INSERT (manufacturer_name)
VALUES (manufacturer_name);

In [0]:
CREATE TABLE IF NOT EXISTS anac_ocorrencias_dev.dimensional.dim_model (
  id_model BIGINT GENERATED ALWAYS AS IDENTITY,
  acft_model STRING,
  acft_icao_type STRING,
  acft_max_gross_weight DOUBLE,
  acft_seats INT,
  acft_description STRING,
  wake_turbulence_category STRING,
  wake_turbulence_group STRING,
  acft_classification STRING,
  engine_type STRING,
  number_of_engines INT,
  id_manufacturer BIGINT,
  CONSTRAINT pk_model PRIMARY KEY (id_model) RELY,
  CONSTRAINT fk_manufacturer FOREIGN KEY (id_manufacturer) REFERENCES anac_ocorrencias_dev.dimensional.dim_manufacturer(id_manufacturer)
)
PARTITIONED BY (wake_turbulence_category,acft_classification,engine_type); 

MERGE INTO anac_ocorrencias_dev.dimensional.dim_model AS tgt
USING (
  WITH cte_model AS (
    SELECT
      LOWER(TRIM(acft_model)) AS acft_model_key,
      LOWER(TRIM(acft_icao_type)) AS acft_icao_key,
      LOWER(TRIM(engine_type)) AS engine_type_key,
      number_of_engines,
      acft_model,
      acft_icao_type,
      acft_max_gross_weight,
      acft_seats,
      acft_description,
      wake_turbulence_category,
      wake_turbulence_group,
      acft_classification,
      engine_type,
      number_of_engines,
      m.id_manufacturer,
      ROW_NUMBER() OVER (
        PARTITION BY LOWER(TRIM(acft_model)), LOWER(TRIM(acft_icao_type)), LOWER(TRIM(engine_type)), number_of_engines
        ORDER BY acft_max_gross_weight DESC NULLS LAST
      ) AS rn
    FROM anac_ocorrencias_dev.gold.transformed_occurrences t
    LEFT JOIN anac_ocorrencias_dev.dimensional.dim_manufacturer m
      ON TRIM(t.acft_manufacturer) = m.manufacturer_name
  )
  SELECT *
  FROM cte_model
  WHERE rn = 1
) AS src
ON LOWER(TRIM(tgt.acft_model)) = src.acft_model_key
AND LOWER(TRIM(tgt.acft_icao_type)) = src.acft_icao_key
AND LOWER(TRIM(tgt.engine_type)) = src.engine_type_key
AND tgt.number_of_engines = src.number_of_engines
WHEN NOT MATCHED THEN
INSERT (
  acft_model, acft_icao_type, acft_max_gross_weight, acft_seats,
  acft_description, wake_turbulence_category, wake_turbulence_group,
  acft_classification, engine_type, number_of_engines, id_manufacturer
)
VALUES (
  acft_model, acft_icao_type, acft_max_gross_weight, acft_seats,
  acft_description, wake_turbulence_category, wake_turbulence_group,
  acft_classification, engine_type, number_of_engines, id_manufacturer
);

In [0]:
CREATE TABLE IF NOT EXISTS anac_ocorrencias_dev.dimensional.dim_aircraft_involved (
  id_aircraft BIGINT GENERATED ALWAYS AS IDENTITY,
  aircraft_call_sign STRING,
  aircraft_damage STRING,
  id_model BIGINT,
  CONSTRAINT pk_aircraft PRIMARY KEY (id_aircraft) RELY,
  CONSTRAINT fk_model FOREIGN KEY (id_model) REFERENCES anac_ocorrencias_dev.dimensional.dim_model(id_model)
)
PARTITIONED BY (aircraft_damage);

MERGE INTO anac_ocorrencias_dev.dimensional.dim_aircraft_involved AS tgt
USING (
  WITH cte_acft_involved AS (
    SELECT DISTINCT
      t.aircraft_call_sign,
      t.aircraft_damage,
      m.id_model,
      ROW_NUMBER() OVER (
        PARTITION BY t.aircraft_call_sign, t.aircraft_damage
        ORDER BY t.aircraft_damage DESC NULLS LAST
      ) AS rn
    FROM anac_ocorrencias_dev.gold.transformed_occurrences t
    LEFT JOIN anac_ocorrencias_dev.dimensional.dim_model m
      ON LOWER(TRIM(t.acft_model)) = LOWER(TRIM(m.acft_model))
     AND LOWER(TRIM(t.acft_icao_type)) = LOWER(TRIM(m.acft_icao_type))
     AND LOWER(TRIM(t.engine_type)) = LOWER(TRIM(m.engine_type))
     AND t.number_of_engines = m.number_of_engines
  )
  SELECT * FROM cte_acft_involved WHERE rn = 1
) AS src
ON tgt.aircraft_call_sign = src.aircraft_call_sign
AND tgt.aircraft_damage = src.aircraft_damage
WHEN NOT MATCHED THEN
INSERT (
  aircraft_call_sign, aircraft_damage, id_model
)
VALUES (
  aircraft_call_sign, aircraft_damage, id_model
);



In [0]:
CREATE TABLE IF NOT EXISTS anac_ocorrencias_dev.dimensional.dim_operation (
  id_operation BIGINT GENERATED ALWAYS AS IDENTITY NOT NULL,
  operated_by STRING,
  operation_phase STRING,
  operation STRING,
  arrival_airport STRING,
  departure_airport STRING,
  CONSTRAINT pk_operation PRIMARY KEY (id_operation) RELY
)
PARTITIONED BY (operation);

MERGE INTO anac_ocorrencias_dev.dimensional.dim_operation AS tgt
USING ( WITH cte_operation AS (
  SELECT DISTINCT 
  operator AS operated_by, 
  operation_phase,
  operation,
  arrival_airport,
  departure_airport,
  ROW_NUMBER() OVER (
    PARTITION BY operator, operation_phase, operation, arrival_airport, departure_airport
    ORDER BY operator DESC 
  ) AS rn
  FROM anac_ocorrencias_dev.gold.transformed_occurrences)

  SELECT * FROM cte_operation WHERE rn = 1
) AS src
ON tgt.operated_by = src.operated_by
AND tgt.operation_phase = src.operation_phase
AND tgt.operation = src.operation
AND tgt.arrival_airport = src.arrival_airport
AND tgt.departure_airport = src.departure_airport
WHEN NOT MATCHED THEN
INSERT (operated_by, operation_phase, operation, arrival_airport, departure_airport)
VALUES (operated_by, operation_phase, operation, arrival_airport, departure_airport);


In [0]:
CREATE TABLE IF NOT EXISTS anac_ocorrencias_dev.dimensional.dim_location (
  id_location BIGINT GENERATED ALWAYS AS IDENTITY,
  city STRING,
  state STRING,
  region STRING,
  icao_airport_code STRING,
  latitude DOUBLE,
  longitude DOUBLE,
  CONSTRAINT pk_location PRIMARY KEY (id_location) RELY
)
PARTITIONED BY (state, region);

MERGE INTO anac_ocorrencias_dev.dimensional.dim_location AS tgt
USING (
  SELECT DISTINCT city, state, region, icao_airport_code, latitude, longitude
  FROM anac_ocorrencias_dev.gold.transformed_occurrences
) AS src
ON tgt.city = src.city
AND tgt.state = src.state
AND tgt.region = src.region
AND tgt.icao_airport_code = src.icao_airport_code
AND tgt.latitude = src.latitude
AND tgt.longitude = src.longitude
WHEN NOT MATCHED THEN
INSERT (city, state, region, icao_airport_code, latitude, longitude)
VALUES (city, state, region, icao_airport_code, latitude, longitude);


In [0]:
CREATE TABLE IF NOT EXISTS anac_ocorrencias_dev.dimensional.dim_time (
  id_time BIGINT GENERATED ALWAYS AS IDENTITY,
  occurrence_date DATE,
  occurrence_time STRING,
  occurrence_year INT,
  occurrence_month STRING,
  day_of_week STRING,
  is_weekend STRING,
  CONSTRAINT pk_time PRIMARY KEY (id_time) RELY
)
PARTITIONED BY (occurrence_month, occurrence_year);

MERGE INTO anac_ocorrencias_dev.dimensional.dim_time AS tgt
USING ( WITH cte_time AS (
  SELECT DISTINCT
    occurrence_date,
    occurrence_time,
    occurrence_year,
    month AS occurrence_month,
    day AS day_of_week,
    is_weekend,
    ROW_NUMBER() OVER (
        PARTITION BY occurrence_date, occurrence_time,occurrence_year, month, day, is_weekend
        ORDER BY occurrence_date DESC NULLS LAST
      ) AS rn
  FROM anac_ocorrencias_dev.gold.transformed_occurrences)

  SELECT * FROM cte_time WHERE rn = 1
) AS src
ON tgt.occurrence_date = src.occurrence_date
AND tgt.occurrence_time = src.occurrence_time
WHEN NOT MATCHED THEN
INSERT (occurrence_date, occurrence_time, occurrence_year, occurrence_month, day_of_week, is_weekend)
VALUES (occurrence_date, occurrence_time, occurrence_year, occurrence_month, day_of_week, is_weekend);


In [0]:
CREATE TABLE IF NOT EXISTS anac_ocorrencias_dev.dimensional.fact_occurrence (
  occurrence_number BIGINT,
  occurrence_classification STRING,
  occurrence_description STRING,
  occurrence_details STRING,
  casualties_crew INTEGER,
  casualties_pax INTEGER,
  casualties_ground INTEGER,
  severe_inj_crew INTEGER,
  severe_inj_pax INTEGER,
  severe_inj_ground INTEGER,
  minor_inj_crew INTEGER,
  minor_inj_pax INTEGER,
  minor_inj_ground INTEGER,
  unharmed_crew INTEGER, 
  unharmed_pax INTEGER,
  total_casualties INTEGER,
  id_aircraft BIGINT,
  id_operation BIGINT,
  id_location BIGINT,
  id_time BIGINT,
  CONSTRAINT pk_occurrence_n PRIMARY KEY (occurrence_number) RELY,
  CONSTRAINT fk_aircraft FOREIGN KEY (id_aircraft) REFERENCES anac_ocorrencias_dev.dimensional.dim_aircraft_involved(id_aircraft),
  CONSTRAINT fk_operation FOREIGN KEY (id_operation) REFERENCES anac_ocorrencias_dev.dimensional.dim_operation(id_operation),
  CONSTRAINT fk_location FOREIGN KEY (id_location) REFERENCES anac_ocorrencias_dev.dimensional.dim_location(id_location),
  CONSTRAINT fk_time FOREIGN KEY (id_time) REFERENCES anac_ocorrencias_dev.dimensional.dim_time(id_time)
)
PARTITIONED BY (occurrence_classification, occurrence_description);

MERGE INTO anac_ocorrencias_dev.dimensional.fact_occurrence AS tgt
USING (
  SELECT
    t.occurrence_number,
    t.occurrence_classification,
    t.occurrence_description,
    t.occurrence_details,
    t.casualties_crew,
    t.casualties_pax,
    t.casualties_ground,
    t.severe_inj_crew,
    t.severe_inj_pax,
    t.severe_inj_ground,
    t.minor_inj_crew,
    t.minor_inj_pax,
    t.minor_inj_ground,
    t.unharmed_crew, 
    t.unharmed_pax,
    t.total_casualties,
    a.id_aircraft,
    o.id_operation,
    l.id_location,
    tm.id_time
  FROM anac_ocorrencias_dev.gold.transformed_occurrences t
  LEFT JOIN anac_ocorrencias_dev.dimensional.dim_aircraft_involved a
    ON t.aircraft_call_sign = a.aircraft_call_sign AND t.aircraft_damage = a.aircraft_damage
  LEFT JOIN anac_ocorrencias_dev.dimensional.dim_operation o
    ON t.operator = o.operated_by AND t.operation_phase = o.operation_phase AND t.operation = o.operation AND t.arrival_airport = o.arrival_airport AND t.departure_airport = o.departure_airport
  LEFT JOIN anac_ocorrencias_dev.dimensional.dim_location l
    ON t.city = l.city AND t.state = l.state AND t.latitude = l.latitude AND t.longitude = l.longitude AND t.region = l.region AND t.icao_airport_code = l.icao_airport_code
  LEFT JOIN anac_ocorrencias_dev.dimensional.dim_time tm
    ON t.occurrence_date = tm.occurrence_date AND t.occurrence_time = tm.occurrence_time AND t.day = tm.day_of_week AND t.is_weekend = tm.is_weekend AND t.occurrence_year = tm.occurrence_year AND t.month = tm.occurrence_month
) AS src
ON tgt.occurrence_number = src.occurrence_number
WHEN NOT MATCHED THEN
INSERT (
    occurrence_number,
    occurrence_classification,
    occurrence_description,
    occurrence_details,
    casualties_crew,
    casualties_pax,
    casualties_ground,
    severe_inj_crew,
    severe_inj_pax,
    severe_inj_ground,
    minor_inj_crew,
    minor_inj_pax,
    minor_inj_ground,
    unharmed_crew, 
    unharmed_pax,
    total_casualties,
    id_aircraft,
    id_operation,
    id_location,
    id_time
)
VALUES (
    src.occurrence_number,
    src.occurrence_classification,
    src.occurrence_description,
    src.occurrence_details,
    src.casualties_crew,
    src.casualties_pax,
    src.casualties_ground,
    src.severe_inj_crew,
    src.severe_inj_pax,
    src.severe_inj_ground,
    src.minor_inj_crew,
    src.minor_inj_pax,
    src.minor_inj_ground,
    src.unharmed_crew, 
    src.unharmed_pax,
    src.total_casualties,
    src.id_aircraft,
    src.id_operation,
    src.id_location,
    src.id_time
);